# 스팀 게임명 보정 + 할인 이력 크롤링

## 조건
- 입력
    - steam_app_names.csv (컬럼: appid, game_name)
    - Windbell 스팀세일 사이트
- 전처리
    - 영문버전으로 게임목록 재크롤링
    - 연령확인 페이지(생년 드롭다운) 통과 처리
    - 보정 실패 시 game_name = '정보 없음' 표기 후 후속 단계에서 제거
- 매칭 규칙
    - Windbell에서 검색명과 사이트 표시명 완전 일치일 때만 할인정보 채택
- 산출
    - 할인율: 원가/할인가 텍스트에서 숫자 추출 후 (원가-할인가)/원가 * 100 (정수 반올림)
    - 할인기간: 페이지 표기값 그대로 저장

## 절차
- 영문명 재크롤링
    1. steam_app_names.csv 읽기 → 각 appid의 스토어 페이지 접속
    2. 연령 확인(생년/월/일 선택) 후 진입 → .apphub_AppName로 공식명 추출
    3. 실패 시 '정보 없음'으로 기록
        - '정보 없음' 행은 후속 분석 대상에서 제거
- 할인 이력 수집
    1. https://steamsale.windbell.co.kr/History?keyword={게임명} 요청
    2. 첫 카드(div.clear-a)에서 사이트 표시명 == 검색명일 때만 채택
    3. 가격 텍스트에서 숫자 추출 → 할인율 계산, 할인기간 추출
    4. 매 게임마다 리스트에 누적, 중간 CSV 저장(복구용)
- AppID 병합
    - 보정된 이름(게임명) 기준으로 원본 appid와 left merge
    - 결과 저장: 최종_할인률.csv

### 1. 영문명 재크롤링

In [ ]:
import os
import pandas as pd
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import Select
from selenium.common.exceptions import NoSuchElementException
import time
from bs4 import BeautifulSoup

# 크롬 옵션 설정
options = webdriver.ChromeOptions()
options.add_argument('--start-maximized')
options.add_experimental_option('detach', True)
options.add_argument("--disable-blink-features=AutomationControlled")
options.add_argument("user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/85.0.4183.102 Safari/537.36")

driver = webdriver.Chrome(options=options)

# CSV 불러오기
df = pd.read_csv('steam_app_names.csv')

# 이름에 '정보'가 포함된 게임만 처리
for i in df[df['game_name'].str.contains('정보')]['appid']:
    url = f'https://store.steampowered.com/app/{i}/'
    driver.get(url)
    time.sleep(1)

    # 연령 확인 처리
    try:
        # 연령 확인 드롭다운이 있는지 확인
        year_select = driver.find_element(By.ID, 'ageYear')

        # 생년월일 선택
        Select(driver.find_element(By.ID, 'ageDay')).select_by_visible_text('1')
        Select(driver.find_element(By.ID, 'ageMonth')).select_by_visible_text('January')
        Select(driver.find_element(By.ID, 'ageYear')).select_by_visible_text('1995')
        time.sleep(0.5)

        # View Page 버튼 클릭
        view_button = driver.find_element(By.CSS_SELECTOR, 'a.btnv6_blue_hoverfade')
        view_button.click()
        time.sleep(1)  # 페이지 이동 대기

    except NoSuchElementException:
        # 연령 확인 페이지가 없으면 그냥 통과
        pass

    # HTML 가져오기
    html = driver.page_source
    soup = BeautifulSoup(html, 'html.parser')

    try:
        name = soup.find('div', class_='apphub_AppName').text.strip()
    except AttributeError:
        name = '정보 없음'

    # 결과 반영
    df.loc[df['appid'] == i, 'game_name'] = name
    print(f"{i} : {name}")


550 : Left 4 Dead 2
1677280 : Company of Heroes 3
1151340 : Fallout 76
1846380 : Need for Speed™ Unbound
3102940 : 정보 없음
1238840 : Battlefield™ 1
3469800 : Company of Heroes 3: Fire & Steel
1254120 : Bless Unleashed
1262580 : Need for Speed™ Payback
335300 : DARK SOULS™ II: Scholar of the First Sin
49520 : Borderlands 2
2016590 : Dark and Darker
1065310 : Evil West
1976440 : CounterSide
1237950 : STAR WARS™ Battlefront™ II
1262600 : Need for Speed™ Rivals
203160 : Tomb Raider
3300460 : Company of Heroes 3: Stealth & Stronghold
3787460 : Warframe: Valkyr Heirloom Collection
1194630 : The Dark Pictures Anthology: Little Hope
680420 : OUTRIDERS
204100 : Max Payne 3
261640 : Borderlands: The Pre-Sequel
35130 : Lara Croft and the Guardian of Light
3658020 : Warframe: Gunblade Battlekit
3816030 : DayZ Badlands
2226680 : Sahara Storms WWIII
1269120 : 정보 없음
3013930 : Hunter's Arena: Revolution
1586750 : Two Strikes
3678320 : 정보 없음
792530 : 정보 없음
3558980 : Warframe: Encore Gemini Collection
355

In [52]:
id = []
for i in df[df['game_name'] == '정보 없음'].index :
    id.append(i)

In [63]:
asd.iloc[id]['게임명'][70:]

16409                                     vertex dispenser
16419                                                brink
16441                         deathspank: thongs of virtue
16470                 serious sam hd: the second encounter
16487                                        rogue warrior
16500                                      dawn of magic 2
16504                                      angle of attack
16507                     watchmen: the end is nigh part 2
16516                                        killing floor
16522                            watchmen: the end is nigh
16525                           f.e.a.r. 2: project origin
16533                                exodus from the earth
16536                                     galactic bowling
16574                        red orchestra: ostfront 41-45
17097                                    crownless duelist
19060    dead or alive 5 last round: core fighters char...
19061    dead or alive 5 last round: core fighters char.

In [66]:
asd.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19426 entries, 0 to 19425
Data columns (total 8 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   appid     19426 non-null  int64 
 1   게임명       19389 non-null  object
 2   출시일       19303 non-null  object
 3   장르        19426 non-null  object
 4   OS        19425 non-null  object
 5   DLC여부     19426 non-null  int64 
 6   평가점수      10618 non-null  object
 7   사용자 평가 수  10618 non-null  object
dtypes: int64(2), object(6)
memory usage: 1.2+ MB


In [67]:
asd.drop(id, inplace=True)

In [68]:
asd.to_csv('스팀게임목록1.csv', index=False)

In [73]:
df.to_csv('최종_영문게임이름.csv', index=False)

### 2. 할인 이력 수집

In [71]:
import os
import pandas as pd
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
import time
from bs4 import BeautifulSoup
import re

# 크롬 옵션 설정
options = webdriver.ChromeOptions()
options.add_argument('--start-maximized')
options.add_experimental_option('detach', True)
options.add_argument("--disable-blink-features=AutomationControlled")
options.add_argument("user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/85.0.4183.102 Safari/537.36")

driver = webdriver.Chrome(options=options)

# 게임 리스트
result_data = []

# 저장 경로 설정
save_path = 'steam_discount_results1.csv'

for name in df['game_name']:
    try:
        url = f'https://steamsale.windbell.co.kr/History?keyword={name}'
        driver.get(url)
        time.sleep(1.5)

        html = driver.page_source
        soup = BeautifulSoup(html, 'html.parser')

        discount_rate = None
        discount_period = None
        price_text = None

        data = soup.find('div', 'clear-a')

        if data:
            # 크롤링된 이름 추출
            data_name_tag = data.find('h6', 'my-1 text-truncate')
            data_name = data_name_tag.text.strip() if data_name_tag else ""

            # 검색한 이름과 동일할 경우에만 크롤링
            if data_name == name:
                # 할인금액 추출 및 할인률 계산
                price_tag = data.find('p', 'my-n1 text-truncate')
                if price_tag:
                    price_text = price_tag.text.strip()
                    prices = re.findall(r'\d[\d,]*', price_text)

                    if len(prices) >= 2:
                        try:
                            original_price = int(prices[0].replace(',', ''))
                            discount_price = int(prices[1].replace(',', ''))
                            if original_price != 0:
                                discount_rate = round(((original_price - discount_price) / original_price) * 100)
                            else:
                                discount_rate = price_text
                        except:
                            discount_rate = price_text
                    else:
                        discount_rate = price_text

                # 할인기간 추출
                period_tag = data.find('p', 'my-n2 text-truncate')
                if period_tag:
                    discount_period = period_tag.text.strip()

                print(f"✅ 크롤링 성공: {name}")
            else:
                print(f"❗ 이름 불일치 - 검색명: {name} / 사이트명: {data_name}")

        else:
            print(f"❗ 데이터 없음: {name}")
        
        # 결과 저장
        result_data.append({
            '게임명': name,
            '할인률': discount_rate,
            '할인기간': discount_period
        })

        # 중간 저장
        df_temp = pd.DataFrame(result_data)
        df_temp.to_csv(save_path, index=False, encoding='utf-8-sig')

    except Exception as e:
        print(f"❌ 오류 발생 - {name}: {e}")
        result_data.append({
            '게임명': name,
            '할인률': None,
            '할인기간': None
        })
        df_temp = pd.DataFrame(result_data)
        df_temp.to_csv(save_path, index=False, encoding='utf-8-sig')

# 최종 출력
df_final = pd.DataFrame(result_data)
df_final

❗ 데이터 없음: GUILTY GEAR -STRIVE-
✅ 크롤링 성공: Magic: The Gathering Arena
✅ 크롤링 성공: EZ2ON REBOOT : R
✅ 크롤링 성공: Sons Of The Forest
✅ 크롤링 성공: Left 4 Dead 2
❗ 이름 불일치 - 검색명: Human Fall Flat / 사이트명: Human Fall Flat VR
✅ 크롤링 성공: Riders Republic
✅ 크롤링 성공: Phasmophobia
✅ 크롤링 성공: Company of Heroes 3
✅ 크롤링 성공: Granblue Fantasy Versus: Rising
✅ 크롤링 성공: Tinkerlands
✅ 크롤링 성공: Dota 2
✅ 크롤링 성공: Crime Simulator
✅ 크롤링 성공: Backrooms: Escape Together
✅ 크롤링 성공: Warhammer 40,000: Darktide
❗ 이름 불일치 - 검색명: World of Tanks / 사이트명: World of Tanks Blitz - Starter Pack
❗ 이름 불일치 - 검색명: Satisfactory / 사이트명: Satisfactory Soundtrack
✅ 크롤링 성공: Unrailed!
❗ 이름 불일치 - 검색명: Monster Hunter: World / 사이트명: Monster Hunter: World - Palico Edit Voucher: Single Voucher
✅ 크롤링 성공: NARAKA: BLADEPOINT
✅ 크롤링 성공: Liar's Bar
✅ 크롤링 성공: A Way Out
✅ 크롤링 성공: The Binding of Isaac: Repentance
✅ 크롤링 성공: Neon Abyss 2
❗ 이름 불일치 - 검색명: Stellaris / 사이트명: Stellaris: Ancient Relics Story Pack
✅ 크롤링 성공: Emissary Zero
✅ 크롤링 성공: Last Epoch
✅ 크롤링 성공: ARK: Surv

,게임명,할인률,할인기간
0,GUILTY GEAR -STRIVE-,None,None
1,Magic: The Gathering Arena,가격 정보가 없습니다.,세일 정보가 없습니다.
2,EZ2ON REBOOT : R,47,2021.03.18 ~ 2025.07.11
3,Sons Of The Forest,66,2023.06.30 ~ 2025.07.11
4,Left 4 Dead 2,90,2017.06.23 ~ 2025.07.11
...,...,...,...
19421,Animals for Clip maker,50,2021.08.12 ~ 2025.07.25
19422,DJMAX RESPECT V - NEXON PACK,80,2021.10.28 ~ 2025.07.11
19423,ARK: Survival Evolved,None,None
19424,Backroom Company,15,2024.11.28 ~


In [79]:
df.columns

Index(['appid', 'game_name'], dtype='object')

In [80]:
df_final.columns

Index(['게임명', '할인률', '할인기간'], dtype='object')

In [81]:
# 먼저 열 이름 맞추기: df의 'game_name'을 '게임명'으로 이름 변경 (임시)
df_renamed = df.rename(columns={'game_name': '게임명'})

# merge 수행 (inner면 매칭되는 것만, left면 df_final 기준으로 유지)
merged_df = df_final.merge(df_renamed[['게임명', 'appid']], on='게임명', how='left')

# 결과 확인
print(merged_df.head())


                          게임명           할인률                     할인기간    appid
0        GUILTY GEAR -STRIVE-          None                     None  1384160
1  Magic: The Gathering Arena  가격 정보가 없습니다.             세일 정보가 없습니다.  2141910
2            EZ2ON REBOOT : R            47  2021.03.18 ~ 2025.07.11  1477590
3            EZ2ON REBOOT : R            47  2021.03.18 ~ 2025.07.11  1477590
4          Sons Of The Forest            66  2023.06.30 ~ 2025.07.11  1326470


In [83]:
merged_df.to_csv('최종_할인률.csv', index=False)